## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease                      
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease       
Reading package lists... Done                                           
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(8, 9):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:19<00:00, 12.54it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_8"])
layer_hs_array.shape

(10000, 267264)

## Layer 8 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

dim_reduced_vecs = layer_hs_array

# random_projector = GaussianRandomProjection(random_state = 42)
# dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 267264)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 267264D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 2.97 s
Outer iteration 0 / 10
  Iteration 19 (68.29 s, search 40.56 s): objective=6107.47 imbalance=1.270 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (131.19 s, search 78.44 s): objective=6111.56 imbalance=1.332 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (193.89 s, search 115.24 s): objective=6116.87 imbalance=1.076 nsplit=0       
Objective improved: keep new clusters
Outer iteration 3 / 10
  Iteration 19 (258.82 s, search 153.47 s): objective=6107.59 imbalance=1.088 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (325.72 s, search 192.94 s): objective=6089.73 imbalance=1.266 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (396.59 s, search 235.17 s): objective=6112.45 imbalance=1.342 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (462.44 s, search 274.10 s): objective=6114

6116.8671875

In [10]:
kmeans.centroids #cluster centers

array([[-0.00737656,  0.00505679,  0.00487592, ...,  0.        ,
         0.        ,  0.        ],
       [-0.0058931 ,  0.00403986,  0.00389536, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01273242,  0.00872835,  0.00841614, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.00787451,  0.00539815,  0.00520507, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01324614,  0.00908052,  0.00875573, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01498067,  0.01026957,  0.00990223, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0000044
1.0000037
1.0000021
1.0000017
1.0000023
1.0000038
0.99999976
1.0000025
1.0000002
1.0000007


In [12]:
kmeans.obj #inertia at each iteration

array([3715.14306641, 6035.81054688, 6084.15673828, 6091.27246094,
       6093.28271484, 6094.45019531, 6095.44482422, 6096.41210938,
       6097.36035156, 6098.12939453, 6098.70849609, 6099.16308594,
       6100.125     , 6101.02587891, 6101.51220703, 6102.29345703,
       6105.02294922, 6107.20361328, 6107.46582031, 6107.46972656,
       3663.89575195, 5994.92919922, 6089.37841797, 6105.73828125,
       6107.72802734, 6108.88574219, 6109.56494141, 6110.21191406,
       6110.66162109, 6110.83789062, 6110.93701172, 6111.00927734,
       6111.06640625, 6111.1484375 , 6111.29833984, 6111.41796875,
       6111.50195312, 6111.54541016, 6111.54833984, 6111.56005859,
       3765.10595703, 6062.36572266, 6097.43408203, 6105.92675781,
       6109.00683594, 6110.4375    , 6111.25244141, 6111.92138672,
       6112.92236328, 6113.73583984, 6114.18603516, 6114.41699219,
       6114.56982422, 6114.69482422, 6114.78271484, 6114.95898438,
       6115.59960938, 6116.73583984, 6116.84960938, 6116.86718

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

6    1389
8    1320
1    1200
7    1146
4    1073
5    1056
0     904
9     775
2     607
3     530
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

6    1389
8    1320
1    1200
7    1146
4    1073
5    1056
0     904
9     775
2     607
3     530
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[6, 6, 3, 8, 2, 8, 6, 2, 6, 8]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[3, 10, 5, 3, 2, 10, 2, 6, 4, 8]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

In [21]:
np.array(max_centroids)

array([6, 6, 6, ..., 8, 8, 8])

In [22]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [23]:
label_to_centroid

{1: 6, 2: 6, 3: 3, 4: 8, 5: 2, 6: 8, 7: 6, 8: 2, 9: 6, 10: 8}

In [24]:
centroid_to_label

{0: 3, 1: 10, 2: 5, 3: 3, 4: 2, 5: 10, 6: 2, 7: 6, 8: 4, 9: 8}

In [25]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 6,  3,  6, ...,  2, 10, 10])

In [26]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.0
Prompt 2 Accuracy:  0.344
Prompt 3 Accuracy:  0.634
Prompt 4 Accuracy:  0.191
Prompt 5 Accuracy:  0.401
Prompt 6 Accuracy:  0.129
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.118
Prompt 9 Accuracy:  0.0
Prompt 10 Accuracy:  0.296


In [27]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.2113


In [28]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.163
Prompt 2 Accuracy:  0.21
Prompt 3 Accuracy:  0.53
Prompt 4 Accuracy:  0.191
Prompt 5 Accuracy:  0.401
Prompt 6 Accuracy:  0.19
Prompt 7 Accuracy:  0.204
Prompt 8 Accuracy:  0.169
Prompt 9 Accuracy:  0.177
Prompt 10 Accuracy:  0.177


In [29]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.2412


## Measure Conversion Accuracy

In [30]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [31]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [32]:
np.mean(max_centroids_to_labels == prompt_ids) #<100%

0.4